# E7

## Read the CSV file

In [2]:
import pandas as pd

data = pd.read_csv('E7.csv')
print(data.head())

C:\Users\Utilisateur\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Utilisateur\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


   Source  Sink  Cost  Supply  Demand
0       1     1    28      20      12
1       1     2     7      20      14
2       1     3    16      20      12
3       1     4     2      20      18
4       1     5    30      20       9


I'm not going to use the csv file but directly the array jsut because it's easier 

In [14]:
import numpy as np

rows = 4  
columns = 6  
matrix = np.zeros((rows, columns))

demands = np.array([12, 14, 12, 18, 9, 65])
supplies = np.array([20, 20, 25, 65])

# Fill the last row with the specified values
matrix[-1, :] = last_row_values
# Fill the last column with the specified values
matrix[:, -1] = last_column_values
print(matrix)

[[ 0.  0.  0.  0.  0. 20.]
 [ 0.  0.  0.  0.  0. 20.]
 [ 0.  0.  0.  0.  0. 25.]
 [12. 14. 12. 18.  9. 65.]]


## NorthWest Corner Method

In [28]:
def northwest_corner_method(cost_matrix, supplies, demands):
    num_rows = len(supplies)
    num_cols = len(demands)
    result = np.zeros((num_rows, num_cols))
    total_cost = 0.0  
    i, j = 0, 0
    while i < num_rows and j < num_cols:
        allocation = min(supplies[i], demands[j])
        result[i, j] = allocation
        total_cost += allocation * cost_matrix[i, j]
        supplies[i] -= allocation
        demands[j] -= allocation
        
        if supplies[i] == 0:
            i += 1
        else:
            j += 1

    return result, total_cost

## Minimum Cost Method

In [34]:
def minimum_cost_method(cost_matrix, supplies, demands):
    # Prepare a matrix to store the transportation plan
    result = np.zeros((3, 5))

    mask = np.zeros_like(cost_matrix, dtype=bool)

    while np.any(demands) and np.any(supplies):
        # Mask the fully allocated cells
        mask |= (supplies[:, None] == 0) | (demands == 0)

        # Find the minimum cost cell that hasn't been fully allocated
        masked_costs = np.where(mask, np.inf, cost_matrix)  # Apply inf where mask is True
        min_cost_index = np.unravel_index(np.argmin(masked_costs), cost_matrix.shape)
        i, j = min_cost_index

        # Allocate as much as possible in the cell with the minimum cost
        allocation = min(supplies[i], demands[j])
        result[i, j] = allocation
        supplies[i] -= allocation
        demands[j] -= allocation

        # Update the mask
        mask[i, j] = True
    total_cost= np.sum(result * cost_matrix)
    return result, total_cost

## Minimum Row Cost Method

In [21]:
def minimum_row_cost_method(cost_matrix, supplies, demands):
    num_rows, num_cols = cost_matrix.shape
    result = np.zeros((num_rows, num_cols))
    total_cost = 0.0

    for i in range(num_rows):
        while supplies[i] > 0 and np.any(demands > 0):
            # Find the minimum cost in the current row that has demand left
            row_costs = np.where(demands > 0, cost_matrix[i], np.inf)
            min_cost_index = np.argmin(row_costs)
            
            # Allocate as much as possible to the cell with the minimum cost
            allocation = min(supplies[i], demands[min_cost_index])
            result[i, min_cost_index] = allocation
            total_cost += allocation * cost_matrix[i, min_cost_index]  # Update total cost
            supplies[i] -= allocation
            demands[min_cost_index] -= allocation

    return result, total_cost

In [22]:
# Function to calculate penalties for a matrix with potential infinities
def calculate_penalties(matrix):
    penalties = []
    for line in matrix:
        # Filter out infinities and sort remaining elements
        filtered = np.sort(line[line != np.inf])
        if len(filtered) > 1:
            penalty = filtered[1] - filtered[0]
        elif len(filtered) == 1:
            penalty = filtered[0]
        else:
            penalty = -np.inf  # No penalty if no available costs
        penalties.append(penalty)
    return np.array(penalties)

## Vogel's Method

In [23]:
def vogels_approximation_method(cost_matrix, supplies, demands):
    num_rows, num_cols = cost_matrix.shape
    result = np.zeros((num_rows, num_cols), dtype=np.float64)
    total_cost = 0.0

    while np.any(demands > 0) and np.any(supplies > 0):
        row_penalties = calculate_penalties(cost_matrix)
        col_penalties = calculate_penalties(cost_matrix.T)

        max_row_penalty_index = np.argmax(row_penalties)
        max_col_penalty_index = np.argmax(col_penalties)

        if row_penalties[max_row_penalty_index] >= col_penalties[max_col_penalty_index]:
            i = max_row_penalty_index
            j = np.argmin(cost_matrix[i, :])
        else:
            j = max_col_penalty_index
            i = np.argmin(cost_matrix[:, j])

        # Allocate as much as possible
        allocation = min(supplies[i], demands[j])
        result[i, j] = allocation
        total_cost += allocation * cost_matrix[i, j]  # Increment total cost
        supplies[i] -= allocation
        demands[j] -= allocation

        # Set the costs to inf where supply or demand is exhausted
        if supplies[i] == 0:
            cost_matrix[i, :] = np.inf
        if demands[j] == 0:
            cost_matrix[:, j] = np.inf

    return result, total_cost

In [87]:
import networkx as nx

def draw_network(current_solution, supplies, demands):
    G = nx.DiGraph()

    num_suppliers, num_demand_points = current_solution.shape
    pos = {}

    # Add nodes with positions
    for i in range(num_suppliers):
        G.add_node(f'S{i+1}')
        pos[f'S{i+1}'] = (1, num_suppliers - i)
    
    for j in range(num_demand_points):
        G.add_node(f'D{j+1}')
        pos[f'D{j+1}'] = (2, num_demand_points - j)

    # Add edges with flows as labels
    for i in range(num_suppliers):
        for j in range(num_demand_points):
            if current_solution[i, j] > 0:
                G.add_edge(f'S{i+1}', f'D{j+1}', weight=current_solution[i, j])

    labels = nx.get_edge_attributes(G, 'weight')
    nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=2000, font_size=15, font_weight='bold')
    nx.draw_networkx_edge_labels(G, pos, edge_labels=labels)
    plt.title('Network Flow Visualization')
    plt.show()

## Transportation Simplex Method

In [92]:
def find_cycle(basis, entering):
    # This function will create a cycle matrix and return the cycle path
    num_rows, num_cols = basis.shape
    marked = np.zeros_like(basis, dtype=bool)
    row_excess = np.array([True] * num_rows)
    col_excess = np.array([True] * num_cols)

    def mark(row, col):
        if marked[row, col]:
            return True
        marked[row, col] = True
        if row_excess[row]:
            for c in range(num_cols):
                if c != col and basis[row, c]:
                    if mark(row, c):
                        return True
        if col_excess[col]:
            for r in range(num_rows):
                if r != row and basis[r, col]:
                    if mark(r, col):
                        return True
        marked[row, col] = False
        return False

    row, col = entering
    mark(row, col)

    path = []
    for r in range(num_rows):
        for c in range(num_cols):
            if marked[r, c]:
                path.append((r, c))
    return path

def pivot_operation(current_solution, costs, basis, supplies, demands, reduced_costs):
    entering = np.unravel_index(np.argmin(reduced_costs), reduced_costs.shape)
    path = find_cycle(basis, entering)
    
    # Determine the maximum amount that can be transferred along the cycle
    amounts = []
    sign = 1
    for idx, pos in enumerate(path):
        if idx % 2 == 1:
            amounts.append(current_solution[pos] * sign)
        sign *= -1
    max_transfer = min(amounts, key=abs)
    
    # Adjust the solution along the cycle
    for idx, pos in enumerate(path):
        if idx % 2 == 1:
            current_solution[pos] -= max_transfer
        else:
            current_solution[pos] += max_transfer

    # Update basis
    basis[path[-1]] = False
    basis[entering] = True

    return current_solution, basis


def transportation_simplex(costs, initial_solution, supplies, demands):
    num_suppliers, num_demand_points = initial_solution.shape
    current_solution = np.array(initial_solution, dtype=float)
    basis = current_solution > 0  # Boolean array to track basic variables
    
    # Check initial feasibility (supplies and demands should match)
    assert sum(supplies) == sum(demands), "Total supply must be equal to total demand!"

    # Main loop of the Transportation Simplex Method
    while True:
        # Step 1: Calculate shadow prices using the current BFS
        u = np.zeros(num_suppliers)    # Shadow prices for suppliers
        v = np.zeros(num_demand_points) # Shadow prices for demand points
        v[-1] = 0  # Set the last shadow price of demand to 0 for normalization
        
        # Calculate u and v based on the current solution and costs
        for i in range(num_suppliers):
            for j in range(num_demand_points):
                if current_solution[i, j] > 0:  # Basic variable
                    if u[i] == 0:
                        u[i] = costs[i, j] - v[j]
        
        for j in range(num_demand_points):
            for i in range(num_suppliers):
                if current_solution[i, j] > 0 and v[j] == 0:
                    v[j] = costs[i, j] - u[i]

        # Step 2: Calculate reduced costs for non-basic variables
        reduced_costs = np.zeros_like(current_solution)
        for i in range(num_suppliers):
            for j in range(num_demand_points):
                if current_solution[i, j] == 0:  # Non-basic variable
                    reduced_costs[i, j] = costs[i, j] - (u[i] + v[j])
        
        # Check for optimality
        if np.all(reduced_costs >= 0):
            print("Optimal solution found!")
            break
        
        # Step 3: Pivot operation to adjust the current solution
        current_solution = pivot_operation(current_solution, costs, basis, supplies, demands, reduced_costs)

    return np.sum(current_solution * costs), current_solution

## Test Northwest Corner Method

In [49]:
cost_matrix = np.array([
    [28, 7, 16, 2, 30],
    [18, 8, 14, 4, 20],
    [10, 12, 13, 5, 28]
])


supply = np.array([20, 20, 25])
demand = np.array([12, 14, 12, 18, 9])

nw_result, nw_cost = northwest_corner_method(cost_matrix, supply, demand)
print("Northwest Corner Method Result:\n", nw_result)
print("Total Cost:", nw_cost)   

Northwest Corner Method Result:
 [[12.  8.  0.  0.  0.]
 [ 0.  6. 12.  2.  0.]
 [ 0.  0.  0. 16.  9.]]
Total Cost: 948.0


## Test Minimum Cost Method

In [51]:
cost_matrix = np.array([
    [28, 7, 16, 2, 30],
    [18, 8, 14, 4, 20],
    [10, 12, 13, 5, 28]
])

supply = np.array([20, 20, 25])
demand = np.array([12, 14, 12, 18, 9])
mc_result, mc_cost = minimum_cost_method(cost_matrix, supply, demand)
print("Minimum Cost Method Result:\n", mc_result)
print("Total Cost:", mc_cost)

Minimum Cost Method Result:
 [[ 0.  2.  0. 18.  0.]
 [ 0. 12.  0.  0.  8.]
 [12.  0. 12.  0.  1.]]
Total Cost: 610.0


## Test Minimum Row Cost Method

In [52]:
cost_matrix = np.array([
    [28, 7, 16, 2, 30],
    [18, 8, 14, 4, 20],
    [10, 12, 13, 5, 28]
])

supply = np.array([20, 20, 25])
demand = np.array([12, 14, 12, 18, 9])
mrc_result, mrc_cost = minimum_row_cost_method(cost_matrix, supply, demand)
print("Minimum Row Cost Method Result:\n", mrc_result)
print("Total Cost:", mrc_cost)

Minimum Row Cost Method Result:
 [[ 0.  2.  0. 18.  0.]
 [ 0. 12.  8.  0.  0.]
 [12.  0.  4.  0.  9.]]
Total Cost: 682.0


## Test Vogel's Approximation Method

In [47]:
cost_matrix = np.array([
    [28, 7, 16, 2, 30],
    [18, 8, 14, 4, 20],
    [10, 12, 13, 5, 28]
], dtype=np.float64)


supplies = np.array([20, 20, 25])
demands = np.array([12, 14, 12, 18, 9])

# Output the results
vogel_result, vogel_cost = vogels_approximation_method(cost_matrix, supplies, demands)
print("Vogel's Approximation Method Result:\n", vogel_result)
print("Total Cost:", vogel_cost)

Vogel's Approximation Method Result:
 [[ 0. 14.  1.  5.  0.]
 [ 0.  0. 11.  0.  9.]
 [12.  0.  0. 13.  0.]]
Total Cost: 643.0


## Test Transportation Simplex Method

In [93]:
cost_matrix = np.array([
    [28, 7, 16, 2, 30],
    [18, 8, 14, 4, 20],
    [10, 12, 13, 5, 28]
], dtype=np.float64)


supplies = np.array([20, 20, 25])
demands = np.array([12, 14, 12, 18, 9])
simplex_cost,simplex_result = transportation_simplex(cost_matrix,mc_result, supplies, demands)  
print("Transportation Simplex Method Result:\n", simplex_result)
print("Total Cost:", simplex_cost)

Optimal solution found!
Transportation Simplex Method Result:
 [[ 0.  2.  0. 18.  0.]
 [ 0. 12.  0.  0.  8.]
 [12.  0. 12.  0.  1.]]
Total Cost: 610.0
